# Benchmarking T5 summarization

Base on code from https://github.com/huggingface/notebooks/blob/master/examples/summarization.ipynb

In [1]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    GIT_ROOT = 'https://github.com/furyhawk/text_summarization/raw/master'
    os.system(f'wget {GIT_ROOT}/notebooks/setup.py')

%run -i setup.py


You are working on a local system.
Files will be searched relative to "..".


In [2]:
%run "$BASE_DIR/settings.py"

%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'png'

# to print output of all statements and not just the last
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# otherwise text between $ signs will be interpreted as formula and printed in italic
pd.set_option('display.html.use_mathjax', False)

# path to import blueprints packages
sys.path.append(BASE_DIR + '/packages')

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets as well as other dependencies. Uncomment the following cell and run it.

In [3]:

#! pip install datasets transformers rouge-score nltk

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [4]:
from huggingface_hub import notebook_login

#notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [5]:
# !apt install git-lfs

In [6]:
import os.path
import pandas as pd
import numpy as np

## Load BBC News summary dataset

In [7]:
root_path = f'../data/BBC News Summary'


# root_path = f'/kaggle/input/bbc-news-summary/BBC News Summary'


def loadDataset(root_path):

    types_of_articles = ['business',
                         'entertainment', 'politics', 'sport', 'tech']
    df = pd.DataFrame(columns=['title', 'article', 'summary'])

    for type_of_article in types_of_articles:
        # type_of_article = 'business'  # entertainment, politices, sport, tech
        num_of_article = len(os.listdir(
            f"{root_path}/News Articles/{type_of_article}"))

        print(f'"Reading {type_of_article} articles"')
        dataframe = pd.DataFrame(columns=['title', 'article', 'summary'])

        for i in tqdm(range(num_of_article)):
            with open(f'{root_path}/News Articles/{type_of_article}/{(i+1):03d}.txt', 'r', encoding="utf8", errors='ignore') as f:
                article = f.read().partition("\n")
            with open(f'{root_path}/Summaries/{type_of_article}/{(i+1):03d}.txt', 'r', encoding="utf8", errors='ignore') as f:
                summary = f.read()

            dataframe.loc[i] = [article[0], article[2].replace(
                '\n', ' ').replace('\r', ''), summary]

        df = df.append(dataframe, ignore_index=True)

    return df

In [8]:
fname = 'bbc.csv'

if os.path.isfile(fname):
    df = pd.read_csv(fname)
else:
    df = loadDataset(root_path)
    df.to_csv(fname, index=False)

### Sample output

In [9]:

sample_text = df['title'][0]
print( 'Title' + sample_text)
sample_text = df['article'][0]
print( 'article' + sample_text)
sample_text = df['summary'][0]
print( 'summary' + sample_text)

TitleAd sales boost Time Warner profit
article Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.  The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.  Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to

In [10]:
df.head()

,title,article,summary,category
0,Ad sales boost Time Warner profit,"Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier. The firm, which is now one of the biggest investors in Goo...","TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn.For the full-year, TimeWarner posted a profit of $3.36bn, up 27% from its 2003 performance, while revenues grew 6.4% to $42.09b...",business
1,Dollar gains on Greenspan speech,The dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said the US trade deficit is set to stabilise. And Alan Greenspan highlighted the US g...,The dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said the US trade deficit is set to stabilise.China's currency remains pegged to the dol...,business
2,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yukos are to ask the buyer of its former production unit to pay back a $900m (£479m) loan. State-owned Rosneft bought the Yugansk unit for $9.3bn in a s...,Yukos' owner Menatep Group says it will ask Rosneft to repay a loan that Yugansk had secured on its assets.State-owned Rosneft bought the Yugansk unit for $9.3bn in a sale forced by Russia to part...,business
3,High fuel prices hit BA's profits,"British Airways has blamed high fuel prices for a 40% drop in profits. Reporting its results for the three months to 31 December 2004, the airline made a pre-tax profit of £75m ($141m) compared ...","Rod Eddington, BA's chief executive, said the results were ""respectable"" in a third quarter when fuel costs rose by £106m or 47.3%.To help offset the increased price of aviation fuel, BA last year...",business
4,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Domecq have risen on speculation that it could be the target of a takeover by France's Pernod Ricard. Reports in the Wall Street Journal and the Financia...,"Pernod has reduced the debt it took on to fund the Seagram purchase to just 1.8bn euros, while Allied has improved the performance of its fast-food chains.Shares in UK drinks and food firm Allied ...",business


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     2225 non-null   object
 1   article   2225 non-null   object
 2   summary   2225 non-null   object
 3   category  2225 non-null   object
dtypes: object(4)
memory usage: 69.7+ KB


In [11]:
#df.to_csv('bbc.csv')

You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/seq2seq).

# Benchmarking models on a summarization task

In this notebook, we will benchmark models for a summarization task. We will use the [BBC dataset](https://www.kaggle.com/pariza/bbc-news-summary) which contains BBC News Summary with only the validation set.


Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [12]:
import transformers

print(transformers.__version__)

4.11.3


## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [13]:
from datasets import Dataset, DatasetDict, load_metric
raw_datasets = Dataset.from_pandas(df)  # Load from dataframe created earlier

metric = load_metric("rouge")

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set:

In [14]:
raw_datasets

Dataset({
    features: ['title', 'article', 'summary', 'category'],
    num_rows: 2225
})

In [15]:
raw_datasets.info 

DatasetInfo(description='', citation='', homepage='', license='', features={'title': Value(dtype='string', id=None), 'article': Value(dtype='string', id=None), 'summary': Value(dtype='string', id=None), 'category': Value(dtype='string', id=None)}, post_processed=None, supervised_keys=None, task_templates=None, builder_name=None, config_name=None, version=None, splits=None, download_checksums=None, download_size=None, post_processing_size=None, dataset_size=None, size_in_bytes=None)

Whether to use news headline or summary as label

## Splitting the dataset in train and test split
We will be using the test set for benchmarking purpose

In [17]:
# 90% train, 10% test + validation
train_testvalid = raw_datasets.train_test_split(test_size=0.1)

raw_datasets = train_testvalid['test']
raw_datasets

Dataset({
    features: ['title', 'article', 'summary', 'category'],
    num_rows: 223
})

To access an actual element, you need to select a split first, then give an index:

You can call its `compute` method with your predictions and labels, which need to be list of decoded strings:

In [18]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [19]:
import nltk
import numpy as np

def compute_piple_metrics(decoded_preds, decoded_labels):
        
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    return {k: round(v, 4) for k, v in result.items()}

In [20]:
from transformers import pipeline
nltk.download('punkt')
"""Loads the model"""
# Initialize the HuggingFace summarization pipeline
summarizer = pipeline("summarization")
model = summarizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\furyx\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

'Loads the model'

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


In [102]:
def transformer_summary(model, text, min_length=3, max_length=50):
    """default huggingface transformer pipeline summary task"""
    assert model
    summarized = model(
        text, min_length=min_length, max_length=max_length)
    # print(summarized)
    return summarized[0]["summary_text"]


In [72]:
raw_datasets['article'][0]
summary = transformer_summary(model, raw_datasets['article'][0])

" Hereford win 3-1 on penalties.  Rovers took the lead on 59 minutes as Michael McIndoe's corner found Adriano Rigoglioso, whose stunning volley soared into the top right of the net. Danny Williams equalised for the hosts after 72 minutes with a crisp finish from just insist the penalty box. McIndoe saw an extra-time penalty saved, giving Mills the chance to net the winning penalty.  Mawson, Travis, Mkandawire, James, Robinson, Daniel Williams, Stanley, Hyde (Pitman 105), Purdie (Mills 83), Brown, Stansfield (Green 102). Subs Not Used: Smith, Scott.  James.  Daniel Williams 72.  Ingham, Mulligan, Fenton, Morley, Priet (Ryan 79), McIndoe, Ravenhill (Price 90), Rigoglioso, Tierney (Jackson 79), Beardsley, Coppinger. Subs Not Used: Warrington, Maloney.  Rigoglioso 59.  1,375  T Parkes (W Midlands). "

Your max_length is set to 512, but you input_length is only 231. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


[{'summary_text': ' Adriano Rigoglioso gave Rovers the lead on 59 minutes with a stunning volley . Danny Williams equalised for the hosts with a crisp finish from just outside the penalty box . Michael McIndoe saw an extra-time penalty saved, giving Mills the chance to net the winner .'}]


In [73]:
summary

' Adriano Rigoglioso gave Rovers the lead on 59 minutes with a stunning volley . Danny Williams equalised for the hosts with a crisp finish from just outside the penalty box . Michael McIndoe saw an extra-time penalty saved, giving Mills the chance to net the winner .'

In [98]:
compute_piple_metrics([summary], [raw_datasets['summary'][0]])

{'rouge1': 4.2105, 'rouge2': 0.0, 'rougeL': 4.2105, 'rougeLsum': 4.2105}

## compute_piple_metrics for default summarization pipeline

In [107]:
metrics = []
for row in raw_datasets:
    article = ''
    article = row['article'][:512] if len(
        row['article']) > 512 else row['article']

    summary = transformer_summary(model, article, max_length=len(article))

    metrics.append(compute_piple_metrics([summary], [row['summary']]))


Your max_length is set to 512, but you input_length is only 130. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 512, but you input_length is only 108. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 512, but you input_length is only 121. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 512, but you input_length is only 111. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 512, but you input_length is only 119. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 512, but you input_length is only 105. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 512, but you input_length is only 113. You might c

## Default pipeline metric

In [109]:
from numpy import array

fmeasure = array([k['rougeL'] for k in metrics]).mean()
print('Default transformer fmeasure: ' + str(fmeasure))


Default transformer fmeasure: 24.261487892376678


In [ ]:
# metrics = updated_dataset.map(compute_metrics)

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that the model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [21]:
model_checkpoint = "t5-base"

In [22]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint) # t5-base

By default, the call above will use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library.

You can directly call this tokenizer on one sentence or a pair of sentences:

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

Instead of one sentence, we can pass along a list of sentences:

To prepare the targets for our model, we need to tokenize them inside the `as_target_tokenizer` context manager. This will make sure the tokenizer uses the special tokens corresponding to the targets:

If you are using one of the five T5 checkpoints we have to prefix the inputs with "summarize:" (the model can also translate and it needs the prefix to know which task it has to perform).

In [23]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model. The padding will be dealt with later on (in a data collator) so we pad examples to the longest length in the batch and not the whole dataset.

In [24]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    # x var for summarization. Column article.
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets y. We are using column title as label.
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

To apply this function on all the pairs of sentences in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [25]:
print(raw_datasets.num_rows)

223


In [26]:

tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

100%|██████████| 1/1 [00:00<00:00,  8.70ba/s]


Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Benchmarking the models

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the `AutoModelForSeq2SeqLM` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [27]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

## Benchmarking T5-base fine tuned model

In [28]:
def finetuned_summary(text, min_length=12, max_length=512):
        model = AutoModelForSeq2SeqLM.from_pretrained(
            "furyhawk/t5-base-finetuned-bbc")
        tokenizer = AutoTokenizer.from_pretrained(
            "furyhawk/t5-base-finetuned-bbc")

        # T5 uses a max_length of 512 so we cut the article to 512 tokens.
        inputs = tokenizer("summarize: " + text,
                           return_tensors="pt", max_length=512, truncation=True)
        outputs = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs['attention_mask'], max_length=max_length,
                                 min_length=min_length, length_penalty=0.1, num_beams=4, early_stopping=True)
        
        summary = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        return summary

In [29]:
summary = finetuned_summary(raw_datasets['article'][0])
summary


Downloading: 100%|██████████| 850M/850M [00:40<00:00, 21.9MB/s]


['Assistant information commissioner Phil Jones said it was classed as marketing if political parties telephoned people to encourage them to vote for them.He said the party always asked people if they would be happy to be contacted again and if they said no, they were not rung again.But seven million telephone numbers are on the Telephone Preference Service (TPS) lists, which ban unsolicited sales and marketing calls.But that does not mean Labour and the Tories are not calling people signed up to the TPS.But "classic market research", such as a poll of voter intentions, did not constitute direct marketing, he']

In [30]:
compute_piple_metrics(summary, [raw_datasets['summary'][0]])


{'rouge1': 46.1538, 'rouge2': 35.567, 'rougeL': 36.4103, 'rougeLsum': 40.0}

In [31]:
metrics = []
for row in raw_datasets:
    article = ''
    article = row['article'][:512] if len(
        row['article']) > 512 else row['article']

    summary = finetuned_summary(article, max_length=len(article))
    # print(summary)

    metrics.append(compute_piple_metrics(summary, [row['summary']]))


T5-base fine tuned metrics
1 run: 30.69969686098655
2 run: 37.009570852017944

In [39]:
fmeasure = array([k['rougeL'] for k in metrics]).mean()
print('finetuned_summary fmeasure: ' + str(fmeasure))


finetuned_summary fmeasure: 37.009570852017944


## Benchmarking T5-base baseline model

In [115]:
def t5_summary(text, min_length=12, max_length=512):
        model = AutoModelForSeq2SeqLM.from_pretrained(
            "t5-base")
        tokenizer = AutoTokenizer.from_pretrained(
            "t5-base")

        # T5 uses a max_length of 512 so we cut the article to 512 tokens.
        inputs = tokenizer("summarize: " + text,
                           return_tensors="pt", max_length=512, truncation=True)
        outputs = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs['attention_mask'], max_length=max_length,
                                 min_length=min_length, length_penalty=0.1, num_beams=4, early_stopping=True)
        
        summary = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        return summary

In [116]:
metrics = []
for row in raw_datasets:
    article = ''
    article = row['article'][:512] if len(
        row['article']) > 512 else row['article']

    summary = t5_summary(article, max_length=len(article))
    # print(summary)

    metrics.append(compute_piple_metrics(summary, [row['summary']]))

In [117]:
fmeasure = array([k['rougeL'] for k in metrics]).mean()
print('t5_summary fmeasure: ' + str(fmeasure))

t5_summary fmeasure: 20.595196412556053
